# 9. 데이터 수집과 그룹

* 하나 이상의 키(함수, 배열, DataFrame 의 칼럼 이름)를 이용해서 pandas 객체를 여러 조각으로 나누는 방법
* 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약통계를 계산하는 방법
* DataFrame의 각 칼럼에 다양한 함수를 적용하는 방법
* 정규화, 선형 회귀, 등급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법
* 피벗 테이블과 교차일람표를 구하는 방법
* 변위치 분석과 다른 데이터 파생 집단 분석을 수행하는 방법

## 9.1. GroupBy 메카닉

그룹의 색인은 다양한 형태가 될 수 있으며, 모두 같은 타입일 필요도 없다.

* 그룹으로 묶을 축과 같은 길이의 리스트나 배열
* DataFrame 의 칼럼 이름을 지칭하는 값
* 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series 객체
* 축 색인 혹은 색인 내의 개별 이름에 대해 실행하는 함수

In [1]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np

In [2]:
df = DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
               'key2': ['one', 'two', 'one', 'two', 'one'],
               'data1': np.random.randn(5),
               'data2': np.random.randn(5)})

In [3]:
grouped = df['data1'].groupby(df['key1'])

In [4]:
grouped

In [5]:
grouped.mean()

key1
a   -0.045371
b   -0.407225
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [7]:
means

key1  key2
a     one    -0.240471
      two     0.344828
b     one    -0.565196
      two    -0.249253
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,-0.240471,0.344828
b,-0.565196,-0.249253


In [9]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [10]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [11]:
df['data1'].groupby([states, years]).mean()

California  2005    0.344828
            2006   -0.565196
Ohio        2005   -0.166965
            2006   -0.396265
Name: data1, dtype: float64

In [12]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.045371,0.312545
b,-0.407225,0.698956


groupby('key1').mean() 에서 key2 가 빠져있는 것은 숫자 데이터가 아니기 때문이다

In [13]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.240471  0.565193
     two   0.344828 -0.192752
b    one  -0.565196  0.095555
     two  -0.249253  1.302357

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 9.1.1. 그룹 간 순회하기

In [15]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -0.084678  0.183583    a  one
1  0.344828 -0.192752    a  two
4 -0.396265  0.946803    a  one
b
      data1     data2 key1 key2
2 -0.565196  0.095555    b  one
3 -0.249253  1.302357    b  two


In [16]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -0.084678  0.183583    a  one
4 -0.396265  0.946803    a  one
('a', 'two')
      data1     data2 key1 key2
1  0.344828 -0.192752    a  two
('b', 'one')
      data1     data2 key1 key2
2 -0.565196  0.095555    b  one
('b', 'two')
      data1     data2 key1 key2
3 -0.249253  1.302357    b  two


In [17]:
pieces = dict(list(df.groupby('key1')))

In [18]:
pieces['b']

,data1,data2,key1,key2
2,-0.565196,0.095555,b,one
3,-0.249253,1.302357,b,two


In [19]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

groupby 메서드는 기본적으로 axis=0 에 대해서 그룹을 만드는데, 다른 축으로 그룹을 만드는 것도 가능하다.

In [20]:
grouped = df.groupby(df.dtypes, axis=1)

In [21]:
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -0.084678  0.183583
 1  0.344828 -0.192752
 2 -0.565196  0.095555
 3 -0.249253  1.302357
 4 -0.396265  0.946803, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### 9.1.2. 칼럼 또는 칼럼의 일부만 선택하기

In [22]:
df.groupby('key1')['data1']

In [23]:
df.groupby('key1')[['data2']]

In [24]:
df['data1'].groupby(df['key1'])

In [25]:
df['data2'].groupby(df['key1'])

In [26]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.565193
     two  -0.192752
b    one   0.095555
     two   1.302357

색인으로 얻는 객체는 groupby 메서드에 전달되는 파라메터에 따라 다르게 된다.
* 리스트나 배열을 넘기면? DataFrameGroupBy 객체
* 단일 값을 넘기면? SeriesGroupBy 객체

In [27]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [28]:
s_grouped

In [29]:
s_grouped.mean()

key1  key2
a     one     0.565193
      two    -0.192752
b     one     0.095555
      two     1.302357
Name: data2, dtype: float64

### 9.1.3. 사전과 Series에서 묶기

In [30]:
people = DataFrame(np.random.randn(5, 5),
                  columns=['a', 'b', 'c', 'd', 'e'],
                  index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [31]:
people.ix[2:3, ['b', 'c']] = np.nan

In [32]:
people

,a,b,c,d,e
Joe,1.160706,-0.406710,-0.777133,1.056040,-0.943203
Steve,1.338564,0.925835,0.935700,1.163249,0.578802
Wes,1.204793,NaN,NaN,-1.111904,0.593218
Jim,-0.780571,1.605634,-0.723777,1.265143,-0.471578
Travis,1.944288,1.015173,-0.510286,0.027876,-1.359909


In [33]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [34]:
by_column = people.groupby(mapping, axis=1)

In [35]:
by_column.sum()

,blue,red
Joe,0.278907,-0.189207
Steve,2.098949,2.843201
Wes,-1.111904,1.798011
Jim,0.541366,0.353484
Travis,-0.482410,1.599553


In [36]:
map_series = Series(mapping)

In [37]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [38]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 9.1.4. 함수로 묶기

In [39]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.584929,1.198924,-1.500911,1.209279,-0.821564
5,1.338564,0.925835,0.935700,1.163249,0.578802
6,1.944288,1.015173,-0.510286,0.027876,-1.359909


In [40]:
key_list = ['one', 'one', 'one', 'two', 'two']

내부적으로 모두 배열로 변환되므로 함수와 배열, 사전 또는 Series 를 함께 섞어 쓰는 것도 전혀 문제가 되지 않는다.

In [41]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  1.160706 -0.406710 -0.777133 -1.111904 -0.943203
  two -0.780571  1.605634 -0.723777  1.265143 -0.471578
5 one  1.338564  0.925835  0.935700  1.163249  0.578802
6 two  1.944288  1.015173 -0.510286  0.027876 -1.359909

### 9.1.5. 색인 단계로 묶기

In [42]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names=['cty', 'tenor'])

In [43]:
hier_df = DataFrame(np.random.randn(4, 5), columns=columns)

In [44]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.556228 -0.122230  0.647156  0.162893  0.164269
1     -0.307964 -0.412308  0.429280  1.132389 -1.329460
2     -1.098653  0.848602  0.961947 -0.943902 -2.072471
3     -1.071898 -1.478750  0.856762 -0.359496 -0.113659

In [45]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 9.2. 데이터 수집

In [46]:
df

,data1,data2,key1,key2
0,-0.084678,0.183583,a,one
1,0.344828,-0.192752,a,two
2,-0.565196,0.095555,b,one
3,-0.249253,1.302357,b,two
4,-0.396265,0.946803,a,one


In [47]:
grouped = df.groupby('key1')

In [48]:
grouped['data1'].quantile(0.9)

key1
a    0.258927
b   -0.280847
Name: data1, dtype: float64

In [49]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [50]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.741093,1.139555
b,0.315943,1.206802


In [51]:
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.045371  0.312545
     std    0.372107  0.580620
     min   -0.396265 -0.192752
     25%   -0.240471 -0.004584
     50%   -0.084678  0.183583
     75%    0.130075  0.565193
     max    0.344828  0.946803
b    count  2.000000  2.000000
     mean  -0.407225  0.698956
     std    0.223406  0.853338
     min   -0.565196  0.095555
     25%   -0.486210  0.397256
     50%   -0.407225  0.698956
     75%   -0.328239  1.000657
     max   -0.249253  1.302357

최적화된 groupby 메서드

<table align="left">
    <thead>
        <td>함수 이름</td>
        <td>설명</td>
    </thead>
    <tbody>
        <tr>
            <td>count</td>
            <td>Nu 그룹 내에 NA 값이 아닌 수를 반환한다.</td>
        </tr>
        <tr>
            <td>sum</td>
            <td>NA 값이 아닌 값들의 합을 구한다.</td>
        </tr>
        <tr>
            <td>mean</td>
            <td>NA 값이 아닌 값들의 평균 값을 구한다.</td>
        </tr>
        <tr>
            <td>median</td>
            <td>NA 값이 아닌 값들의 산술 중간값을 구한다.</td>
        </tr>
        <tr>
            <td>std, var</td>
            <td>편향되지 않은 (n-1을 분모로 하는) 표준편차와 분산</td>
        </tr>
        <tr>
            <td>min, max</td>
            <td>NA 값이 아닌 값 중 최소 값과 최대 값</td>
        </tr>
        <tr>
            <td>prod</td>
            <td>NA 값이 아닌 값의 곱</td>
        </tr>
        <tr>
            <td>first, last</td>
            <td>NA 값이 아닌 값들 중 첫 번째 값과 마지막 값</td>
        </tr>
    </tbody>
</table>

In [52]:
tips = pd.read_csv('ch08/tips.csv')

In [53]:
tips['tip_pct'] = tips['tip']/tips['total_bill']

In [54]:
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


### 9.2.1. 칼럼에 여러 가지 함수 적용하기

In [55]:
grouped = tips.groupby(['sex', 'smoker'])

In [56]:
grouped_pct = grouped['tip_pct']

기술 통계 함수는 문자열로 이름을 넘겨도 정상적인 수행이 가능하다

In [57]:
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [58]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [59]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [60]:
functions = ['count', 'mean', 'max']

In [61]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [62]:
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [63]:
result['tip_pct']

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

In [64]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [65]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

In [66]:
grouped.agg({'tip': np.max, 'size': 'sum'})

size   tip
sex    smoker            
Female No       140   5.2
       Yes       74   6.5
Male   No       263   9.0
       Yes      150  10.0

In [67]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
            'size': 'sum'})

size   tip_pct                              
               sum       min       max      mean       std
sex    smoker                                             
Female No      140  0.056797  0.252672  0.156921  0.036421
       Yes      74  0.056433  0.416667  0.182150  0.071595
Male   No      263  0.071804  0.291990  0.160669  0.041849
       Yes     150  0.035638  0.710345  0.152771  0.090588

### 9.2.2. 색인되지 않은 형태로 집계된 데이터 반환하기

In [68]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


reset_index 메서드를 호출해 같은 결과를 얻을 수 있다.

In [69]:
tips.groupby(['sex', 'smoker']).mean().reset_index()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


## 9.3. 그룹별 연산과 변형

In [70]:
df

,data1,data2,key1,key2
0,-0.084678,0.183583,a,one
1,0.344828,-0.192752,a,two
2,-0.565196,0.095555,b,one
3,-0.249253,1.302357,b,two
4,-0.396265,0.946803,a,one


In [71]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')

In [72]:
k1_means

,mean_data1,mean_data2
key1,,
a,-0.045371,0.312545
b,-0.407225,0.698956


In [73]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,-0.084678,0.183583,a,one,-0.045371,0.312545
1,0.344828,-0.192752,a,two,-0.045371,0.312545
4,-0.396265,0.946803,a,one,-0.045371,0.312545
2,-0.565196,0.095555,b,one,-0.407225,0.698956
3,-0.249253,1.302357,b,two,-0.407225,0.698956


In [74]:
key = ['one', 'two', 'one', 'two', 'one']

In [75]:
people

,a,b,c,d,e
Joe,1.160706,-0.406710,-0.777133,1.056040,-0.943203
Steve,1.338564,0.925835,0.935700,1.163249,0.578802
Wes,1.204793,NaN,NaN,-1.111904,0.593218
Jim,-0.780571,1.605634,-0.723777,1.265143,-0.471578
Travis,1.944288,1.015173,-0.510286,0.027876,-1.359909


In [76]:
people.groupby(key).mean()

,a,b,c,d,e
one,1.436596,0.304232,-0.643710,-0.009329,-0.569965
two,0.278996,1.265735,0.105961,1.214196,0.053612


In [77]:
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,1.436596,0.304232,-0.643710,-0.009329,-0.569965
Steve,0.278996,1.265735,0.105961,1.214196,0.053612
Wes,1.436596,0.304232,-0.643710,-0.009329,-0.569965
Jim,0.278996,1.265735,0.105961,1.214196,0.053612
Travis,1.436596,0.304232,-0.643710,-0.009329,-0.569965


In [78]:
def demean(arr):
    return arr - arr.mean()

In [79]:
demeaned = people.groupby(key).transform(demean)

In [80]:
demeaned

,a,b,c,d,e
Joe,-0.275890,-0.710941,-0.133424,1.065369,-0.373238
Steve,1.059567,-0.339899,0.829739,-0.050947,0.525190
Wes,-0.231803,NaN,NaN,-1.102575,1.163183
Jim,-1.059567,0.339899,-0.829739,0.050947,-0.525190
Travis,0.507692,0.710941,0.133424,0.037205,-0.789944


In [81]:
demeaned.groupby(key).mean()

,a,b,c,d,e
one,7.401487e-17,0.000000e+00,5.551115e-17,2.312965e-18,0.0
two,0.000000e+00,5.551115e-17,0.000000e+00,0.000000e+00,0.0


### 9.3.1. apply: 분리-적용-병합

In [82]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [83]:
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [84]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [85]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

apply 메서드를 독창적인 방법으로 다양하게 사용할 수 있다.<br/>
넘기는 함수 내부에서 하는 일은 전적으로 구현하기 나름이다. 그저 pandas 객체나 스칼라 값을 반환하는 함수면 된다.

In [86]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [87]:
result

smoker       
No      count    151.000000
        mean       0.159328
        std        0.039910
        min        0.056797
        25%        0.136906
        50%        0.155625
        75%        0.185014
        max        0.291990
Yes     count     93.000000
        mean       0.163196
        std        0.085119
        min        0.035638
        25%        0.106771
        50%        0.153846
        75%        0.195059
        max        0.710345
dtype: float64

In [88]:
result.unstack()

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [89]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill        tip       size    tip_pct
sex    smoker                                                   
Female No     count   54.000000  54.000000  54.000000  54.000000
              mean    18.105185   2.773519   2.592593   0.156921
              std      7.286455   1.128425   1.073146   0.036421
              min      7.250000   1.000000   1.000000   0.056797
              25%     12.650000   2.000000   2.000000   0.139708
              50%     16.690000   2.680000   2.000000   0.149691
              75%     20.862500   3.437500   3.000000   0.181630
              max     35.830000   5.200000   6.000000   0.252672
       Yes    count   33.000000  33.000000  33.000000  33.000000
              mean    17.977879   2.931515   2.242424   0.182150
              std      9.189751   1.219916   0.613917   0.071595
              min      3.070000   1.000000   1.000000   0.056433
              25%     12.760000   2.000000   2.000000   0.152439
              50%     16.270000   2.880000   2.000000   0.173913
              75%     22.120000   3.500000   2.000000   0.198216
              max     44.300000   6.500000   4.000000   0.416667
Male   No     count   97.000000  97.000000  97.000000  97.000000
              mean    19.791237   3.113402   2.711340   0.160669
              std      8.726566   1.489559   0.989094   0.041849
              min      7.510000   1.250000   2.000000   0.071804
              25%     13.810000   2.000000   2.000000   0.131810
              50%     18.240000   2.740000   2.000000   0.157604
              75%     22.820000   3.710000   3.000000   0.186220
              max     48.330000   9.000000   6.000000   0.291990
       Yes    count   60.000000  60.000000  60.000000  60.000000
              mean    22.284500   3.051167   2.500000   0.152771
              std      9.911845   1.500120   0.892530   0.090588
              min      7.250000   1.000000   1.000000   0.035638
              25%     15.272500   2.000000   2.000000   0.101845
              50%     20.390000   3.000000   2.000000   0.141015
              75%     28.572500   3.820000   3.000000   0.191697
              max     50.810000  10.000000   5.000000   0.710345

group_keys=False 옵션을 이용하여 그룹 색인을 생략할 수 있다.

In [90]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### 9.3.2. 변위치 분석과 버킷 분석

In [91]:
frame = DataFrame({'data1': np.random.randn(1000),
                  'data2': np.random.randn(1000)})

In [92]:
factor = pd.cut(frame.data1, 4)

In [93]:
factor[:10]

0      (0.307, 1.938]
1     (-1.325, 0.307]
2    (-2.963, -1.325]
3      (0.307, 1.938]
4      (1.938, 3.569]
5      (0.307, 1.938]
6     (-1.325, 0.307]
7     (-1.325, 0.307]
8     (-1.325, 0.307]
9      (0.307, 1.938]
Name: data1, dtype: category
Categories (4, object): [(-2.963, -1.325] < (-1.325, 0.307] < (0.307, 1.938] < (1.938, 3.569]]

In [94]:
def get_stats(group):
    return {
        'min': group.min(), 'max': group.max(),
        'count': group.count(), 'mean': group.mean()
    }

In [95]:
grouped = frame.data2.groupby(factor)

In [96]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.963, -1.325]",90.0,2.207734,-0.016102,-1.728978
"(-1.325, 0.307]",522.0,3.021811,-0.094408,-3.388408
"(0.307, 1.938]",355.0,3.297427,-0.087491,-2.423983
"(1.938, 3.569]",33.0,1.680837,0.083974,-2.366074


In [97]:
# 변위치 숫자를 반환한다.
grouping = pd.cut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,14.0,1.964674,0.042596,-1.292289
1,32.0,2.144546,-0.081468,-1.728978
2,113.0,2.207734,-0.028742,-1.907815
3,210.0,2.989778,-0.171654,-2.894434
4,243.0,3.021811,-0.038783,-3.388408
5,208.0,2.438174,-0.061868,-2.323151
6,121.0,3.297427,-0.144576,-2.423983
7,42.0,3.029944,0.007057,-2.417441
8,12.0,1.680837,-0.023657,-1.198820


### 9.3.3. 예제: 그룹에 국한된 값으로 누락값 채우기

In [98]:
s = Series(np.random.randn(6))

In [99]:
s[::2] = np.nan

In [100]:
s

0         NaN
1   -1.643789
2         NaN
3   -0.889729
4         NaN
5   -0.481925
dtype: float64

In [101]:
s.fillna(s.mean())

0   -1.005148
1   -1.643789
2   -1.005148
3   -0.889729
4   -1.005148
5   -0.481925
dtype: float64

In [102]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
        'Oregon', 'Nevada', 'California', 'Idaho']

In [103]:
group_key = ['East'] * 4 + ['West'] * 4

In [104]:
data = Series(np.random.randn(8), index=states)

In [105]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [106]:
data

Ohio          0.110327
New York     -0.954388
Vermont            NaN
Florida      -0.820507
Oregon        0.086591
Nevada             NaN
California    0.830370
Idaho              NaN
dtype: float64

In [107]:
data.groupby(group_key).mean()

East   -0.554856
West    0.458480
dtype: float64

In [108]:
fill_mean = lambda g: g.fillna(g.mean())

In [109]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.110327
New York     -0.954388
Vermont      -0.554856
Florida      -0.820507
Oregon        0.086591
Nevada        0.458480
California    0.830370
Idaho         0.458480
dtype: float64

In [110]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          0.110327
New York     -0.954388
Vermont       0.500000
Florida      -0.820507
Oregon        0.086591
Nevada       -1.000000
California    0.830370
Idaho        -1.000000
dtype: float64

### 9.3.4. 예제: 랜덤 표본과 순열

랜덤 표본 추출방법

* np.random.permutation(N) 에서 K 원소를 선택하는 방법

In [112]:
# 하트, 스페이드, 클로버, 다이아몬드
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10]*3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = Series(card_val, index=cards)

In [113]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [114]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])

In [115]:
draw(deck)

2C    2
6D    6
9D    9
5D    5
8H    8
dtype: int64

In [116]:
# 각 무늬별 2장의 카드를 무작위로 뽑고 싶다면?
get_suit = lambda card: card[-1] # 마지막 글자를 뽑는다.

In [117]:
deck.groupby(get_suit).apply(draw, n=2)

C  4C      4
   KC     10
D  10D    10
   4D      4
H  10H    10
   8H      8
S  KS     10
   AS      1
dtype: int64

In [118]:
# 또 다른 방법
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

KC    10
2C     2
5D     5
QD    10
6H     6
4H     4
6S     6
JS    10
dtype: int64

### 9.3.5. 예제: 그룹 가중 평균과 상관관계

In [121]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
               'data': np.random.randn(8),
               'weights': np.random.rand(8)})

In [122]:
df

,category,data,weights
0,a,-0.538576,0.264272
1,a,-0.550269,0.532792
2,a,-0.803973,0.683698
3,a,-1.145193,0.491828
4,b,-0.663079,0.251640
5,b,0.146830,0.374343
6,b,-1.136195,0.870372
7,b,-1.119741,0.212003


In [123]:
grouped = df.groupby('category')

In [124]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [125]:
grouped.apply(get_wavg)

category
a   -0.784969
b   -0.783321
dtype: float64

In [126]:
close_px = pd.read_csv('ch09/stock_px.csv', parse_dates=True, index_col=0)

In [127]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [128]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [129]:
rets = close_px.pct_change().dropna()

In [130]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [131]:
by_year = rets.groupby(lambda x: x.year)

In [132]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [133]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 9.3.6. 예제: 그룹 상의 선형 회귀

계량 경제 라이브러리인 statsmodels 를 사용해서 각 데이터 묶음마다 최소제곱(Ordinary least squares) 으로 회귀를 수행할 수 있다.

In [134]:
import statsmodels.api as sm

In [135]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

In [136]:
by_year.apply(regress,'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 9.4. 피벗 테이블과 교차일람표

피벗 테이블 - 데이터를 하나 이상의 키로 수집해서 어떤 키는 로우에, 어떤 키는 칼럼에 나열해서 데이터를 정렬한다.

In [140]:
tips.pivot_table(['sex', 'smoker'])

ValueError: No group keys passed!

In [139]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'], columns='smoker')

tip_pct                size          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   0.165296  0.209129  2.500000  2.000000
       Sat   0.147993  0.163817  2.307692  2.200000
       Sun   0.165710  0.237075  3.071429  2.500000
       Thur  0.155971  0.163073  2.480000  2.428571
Male   Fri   0.138005  0.144730  2.000000  2.125000
       Sat   0.162132  0.139067  2.656250  2.629630
       Sun   0.158291  0.173964  2.883721  2.600000
       Thur  0.165706  0.164417  2.500000  2.300000

margins=True 옵션을 추가하여 부분합을 포함하도록 확장이 가능하다

In [141]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'], columns='smoker', margins=True)

tip_pct                          size                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   0.165296  0.209129  0.199388  2.500000  2.000000  2.111111
       Sat   0.147993  0.163817  0.156470  2.307692  2.200000  2.250000
       Sun   0.165710  0.237075  0.181569  3.071429  2.500000  2.944444
       Thur  0.155971  0.163073  0.157525  2.480000  2.428571  2.468750
Male   Fri   0.138005  0.144730  0.143385  2.000000  2.125000  2.100000
       Sat   0.162132  0.139067  0.151577  2.656250  2.629630  2.644068
       Sun   0.158291  0.173964  0.162344  2.883721  2.600000  2.810345
       Thur  0.165706  0.164417  0.165276  2.500000  2.300000  2.433333
All          0.159328  0.163196  0.160803  2.668874  2.408602  2.569672

In [142]:
tips.pivot_table('tip_pct', index=['sex', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
sex    smoker                               
Female No       2.0  13.0  14.0  25.0   54.0
       Yes      7.0  15.0   4.0   7.0   33.0
Male   No       2.0  32.0  43.0  20.0   97.0
       Yes      8.0  27.0  15.0  10.0   60.0
All            19.0  87.0  76.0  62.0  244.0

In [143]:
# 만약 어떤 조합이 비어있거나 NA 값이라면?
tips.pivot_table('size', index=['time', 'sex', 'smoker'],
                columns='day', aggfunc='sum', fill_value=0)

day                   Fri  Sat  Sun  Thur
time   sex    smoker                     
Dinner Female No        2   30   43     2
              Yes       8   33   10     0
       Male   No        4   85  124     0
              Yes      12   71   39     0
Lunch  Female No        3    0    0    60
              Yes       6    0    0    17
       Male   No        0    0    0    50
              Yes       5    0    0    23

### 9.4.1. 교차일람표

In [144]:
from io import StringIO
data = """\
Sample Gender Handedness
1 Female Right-handed
2 Male Left-handed
3 Female Right-handed
4 Male Right-handed
5 Male Left-handed
6 Male Right-handed
7 Female Right-handed
8 Female Left-handed
9 Male Right-handed
10 Female Right-handed
"""

data = pd.read_table(StringIO(data), sep='\s+')

In [145]:
data

,Sample,Gender,Handedness
0,1,Female,Right-handed
1,2,Male,Left-handed
2,3,Female,Right-handed
3,4,Male,Right-handed
4,5,Male,Left-handed
5,6,Male,Right-handed
6,7,Female,Right-handed
7,8,Female,Left-handed
8,9,Male,Right-handed
9,10,Female,Right-handed


In [146]:
pd.crosstab(data.Gender, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Gender,,,
Female,1,4,5
Male,2,3,5
All,3,7,10


In [147]:
pd.crosstab([tips.time,tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

## 9.5. 예제: 2012년 연방 선거관리위원회 데이터베이스

예제에 사용된 샘플 데이터는 용량이 커서 첨부하지 않았다.

In [148]:
fec = pd.read_csv('ch09/P00000001-ALL.csv')

/Users/NAVER/Documents/dev/dev_env/Anaconda/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [149]:
fec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001731 entries, 0 to 1001730
Data columns (total 16 columns):
cmte_id              1001731 non-null object
cand_id              1001731 non-null object
cand_nm              1001731 non-null object
contbr_nm            1001731 non-null object
contbr_city          1001712 non-null object
contbr_st            1001727 non-null object
contbr_zip           1001620 non-null object
contbr_employer      988002 non-null object
contbr_occupation    993301 non-null object
contb_receipt_amt    1001731 non-null float64
contb_receipt_dt     1001731 non-null object
receipt_desc         14166 non-null object
memo_cd              92482 non-null object
memo_text            97770 non-null object
form_tp              1001731 non-null object
file_num             1001731 non-null int64
dtypes: float64(1), int64(1), object(14)
memory usage: 122.3+ MB


In [150]:
fec.ix[123456]

cmte_id                             C00431445
cand_id                             P80003338
cand_nm                         Obama, Barack
contbr_nm                         ELLMAN, IRA
contbr_city                             TEMPE
contbr_st                                  AZ
contbr_zip                          852816719
contbr_employer      ARIZONA STATE UNIVERSITY
contbr_occupation                   PROFESSOR
contb_receipt_amt                          50
contb_receipt_dt                    01-DEC-11
receipt_desc                              NaN
memo_cd                                   NaN
memo_text                                 NaN
form_tp                                 SA17A
file_num                               772372
Name: 123456, dtype: object

In [151]:
unique_cands = fec.cand_nm.unique()

In [152]:
unique_cands

array(['Bachmann, Michelle', 'Romney, Mitt', 'Obama, Barack',
       "Roemer, Charles E. 'Buddy' III", 'Pawlenty, Timothy',
       'Johnson, Gary Earl', 'Paul, Ron', 'Santorum, Rick', 'Cain, Herman',
       'Gingrich, Newt', 'McCotter, Thaddeus G', 'Huntsman, Jon',
       'Perry, Rick'], dtype=object)

In [153]:
unique_cands[2]

'Obama, Barack'

In [154]:
parties = {'Bachmann, Michelle': 'Republican',
          'Cain, Herman': 'Republican',
          'Gingrich, Newt': 'Republican',
          'Huntsman, Jon': 'Republican',
          'Johnson, Gary Earl': 'Republican',
          'McCotter, Thaddeus G': 'Republican',
          'Obama, Barack': 'Democrat',
          'Paul, Rick': 'Republican',
          'Pawlenty, Timothy': 'Republican',
          'Perry, Rick': 'Republican',
          "Romer, Charles E. 'Buddy' III": 'Republican',
          'Romney, Mitt': 'Republican',
          'Santorum, Rick': 'Republican'}

In [155]:
fec.cand_nm[123456:123461]

123456    Obama, Barack
123457    Obama, Barack
123458    Obama, Barack
123459    Obama, Barack
123460    Obama, Barack
Name: cand_nm, dtype: object

In [156]:
fec.cand_nm[123456:123461].map(parties)

123456    Democrat
123457    Democrat
123458    Democrat
123459    Democrat
123460    Democrat
Name: cand_nm, dtype: object

In [157]:
fec['party'] = fec.cand_nm.map(parties)

In [158]:
fec['party'].value_counts()

Democrat      593746
Republican    258308
Name: party, dtype: int64

In [160]:
(fec.contb_receipt_amt > 0).value_counts()

True     991475
False     10256
Name: contb_receipt_amt, dtype: int64

In [161]:
fec = fec[fec.contb_receipt_amt > 0]

In [162]:
fec_mrbo = fec[fec.cand_nm.isin(['Obama, Barack', 'Romney, Mitt'])]

### 9.5.1. 직장 및 피고용별 기부 통계

In [163]:
fec.contbr_occupation.value_counts()[:10]

RETIRED                                   233990
INFORMATION REQUESTED                      35107
ATTORNEY                                   34286
HOMEMAKER                                  29931
PHYSICIAN                                  23432
INFORMATION REQUESTED PER BEST EFFORTS     21138
ENGINEER                                   14334
TEACHER                                    13990
CONSULTANT                                 13273
PROFESSOR                                  12555
Name: contbr_occupation, dtype: int64

In [164]:
occ_mapping = {
    'INFORMATION REQUEST PER BEST EFFORTS': 'NOT PROVIDED',
    'INFORMATION REQUESTED': 'NOT PROVIDED',
    'INFORMATION REQUESTED(BEST EFFORT)': 'NOT PROVIDED',
    'C.E.O.': 'CEO'
}

In [170]:
# 맵핑 정보가 없는 직업은 키를 그대로 반환
f = lambda x: occ_mapping.get(x, x)
fec.loc['contbr_occupation'] = fec.contbr_occupation.map(f)

/Users/NAVER/Documents/dev/dev_env/Anaconda/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [167]:
emp_mapping = {
    'INFORMATION REQUEST PER BEST EFFORTS': 'NOT PROVIDED',
    'INFORMATION REQUESTED': 'NOT PROVIDED',
    'SELF': 'SELF-EMPLOYED',
    'SELF EMPLOYED': 'SELF-EMPLOYED'
}

In [168]:
# 맵핑 정보가 없는 직업은 키를 그대로 반환
f = lambda x: emp_mapping.get(x, x)
fec.contbr_employer = fec.contbr_employer.map(f)

/Users/NAVER/Documents/dev/dev_env/Anaconda/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
